In [1]:
import pathlib
import re

import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from implicit import als, lmf
from scipy.sparse import csr_matrix

In [2]:
def load_reviews_dataset(folder_with_csvs_path="../data/raw/steam-reviews-dataset/"):
    # steam reviews dataset
    chunk_paths = [path for path in pathlib.Path(folder_with_csvs_path).glob("*.csv")]
    dfs = []
    for i in chunk_paths:
        dfs.append(pd.read_csv(i))
    return pd.concat(dfs).rename(columns={"appid": "app_id", "steamid": "user_id"})


def load_game_recommendations_on_steam_games_dataset(csv_path="../data/raw/game-recommendations-on-steam/games.csv"):
    return (
        pd.read_csv(csv_path)
        .rename(columns={
            "date_release": "release_date",
            "price_final": "final_price",
            "price_original": "original_price",
            "win": "win_support",
            "mac": "mac_support",
            "linux": "linux_support",
            "steam_deck": "steam_deck_support",
        }).drop("discount", axis=1)
    )


def load_steam_games_complete_dataset(csv_path="../data/raw/steam-games-complete-dataset/steam_games.csv"):
    def convert_price_to_float(x):
        if isinstance(x, float):
            return x
        elif x.startswith("$"):
            return float(x.lstrip("$"))
        elif x == "Free":
            return 0
        else:
            return None

    # int(x.lstrip("$") if x.startswith("$") else (if )

    df_games_complete = (
        pd.read_csv(csv_path)
        .query("types == 'app' | types == 'bundle'")
        .rename(columns={"name": "title", "achievements": "n_game_achievements"})
        .assign(refined_price=lambda df: df["original_price"].apply(convert_price_to_float))
    )
    df_games_complete["app_id"] = (
        df_games_complete.url
        .str.removeprefix("https://store.steampowered.com/")
        .str.split("/")
        .apply(lambda x: x[1]).astype(int)
    )
    df_games_complete["app_id"] = df_games_complete["app_id"].astype(int)
    return df_games_complete


def load_dataset(
    steam_reviews_dataframe,
    game_rec_steam_dataframe,
    steam_games_complete_dataframe,
):
    # @TODO: remove duplicates
    duplicates = (("name", "title"), ("release_date", "date_release"), ("rating", "positive_ratio", "recent_reviews"))
    games_joined = (
        steam_games_complete_dataframe.set_index("app_id")
        .join(game_rec_steam_dataframe.set_index("app_id"), lsuffix="_sgcd", rsuffix="_grps")
        .drop([
            "url",
            "minimum_requirements", "recommended_requirements",  # can be turned into features
            # can be turned into features
            "discount_price",
            "user_reviews", "rating", "positive_ratio",  # can result in data leakage
            "final_price", "original_price_grps",
        ], axis=1)
        .assign(
            final_title=lambda df: df.apply(
                lambda row: (row.title_grps if not pd.isna(row.title_grps) else row.title_sgcd),
                axis=1),
        )
    )
    return (
        steam_reviews_dataframe.sort_values(by="app_id")
        .join(games_joined.sort_values(by="app_id"), on="app_id", lsuffix="srd")
    )

In [3]:
reviews = load_reviews_dataset()

In [4]:
df_games_rec = load_game_recommendations_on_steam_games_dataset()

In [5]:
games_complete = load_steam_games_complete_dataset()

In [6]:
ready_dataset = load_dataset(reviews, df_games_rec, games_complete)

## Missing imputation

In [7]:
for column, n_na in (
    ready_dataset.isna().mean()
    .where(lambda x: x > 0).dropna()
    .sort_values().items()):
    print(f"Column: \033[1m{column}\033[0m, missing values percentage: \033[1m{(n_na * 100):.2f}%\033[0m")

Column: review, missing values percentage: 0.15%
Column: languages, missing values percentage: 2.73%
Column: publisher, missing values percentage: 2.73%
Column: developer, missing values percentage: 2.73%
Column: final_title, missing values percentage: 2.73%
Column: desc_snippet, missing values percentage: 2.73%
Column: title_sgcd, missing values percentage: 2.73%
Column: types, missing values percentage: 2.73%
Column: genre, missing values percentage: 2.74%
Column: game_details, missing values percentage: 2.91%
Column: original_price_sgcd, missing values percentage: 3.83%
Column: game_description, missing values percentage: 3.91%
Column: popular_tags, missing values percentage: 3.91%
Column: all_reviews, missing values percentage: 3.91%
Column: release_date_sgcd, missing values percentage: 3.92%
Column: title_grps, missing values percentage: 5.61%
Column: release_date_grps, missing values percentage: 5.61%
Column: win_support, missing values percentage: 5.61%
Column: mac_support, miss

Imputation-ready columns:
* Languages – unknown as additional category (dropped on training)
* Publisher – uknown as additional category
* Developer – uknown as additional category
* types – transform into columns

* win_support – KNN imputer based on other games?
* mac_support – KNN imputer based on other games?
* linux_support - KNN imputer based on other games?

* release_date_grps ? release_date_sgcd join 

* refined_price – mean price or median based on distribution
* n_game_archievements - mean price or median based on distribution
* mature content – missing values category

In [8]:
languages_categories_to_include = ['English', 'French', 'Italian', 'German', 'Spanish - Spain',
       'Japanese', 'Polish', 'Portuguese - Brazil', 'Russian',
       'Traditional Chinese', 'Korean', 'Simplified Chinese', 'Arabic',
       'Portuguese', 'Turkish', 'Thai', 'Ukrainian', 'Czech',
       'Spanish - Latin America', 'Dutch', 'Hungarian', 'Danish',
       'Finnish', 'Norwegian', 'Swedish', 'Romanian', 'Greek',
       'Bulgarian','Vietnamese', "Slovakian"]

genres_to_include = ['Action', 'Adventure', 'Massively Multiplayer', 'Strategy',
       'Free to Play', 'RPG', 'Indie', 'Early Access', 'Simulation',
       'Racing', 'Casual', 'Sports', 'Violent', 'Gore', 'Valve', 'Nudity', 
       'Sexual Content', 'Education', 'Video Production', 'Short']

### DROP OTHER GENRES

In [ ]:
# if i don't want to make separate category for uknown genre or languages, then I can remove 
# ready_dataset["languages"].fillna([])
# ready_dataset["genre"].fillna([])

In [33]:
# for language in languages_categories_to_include:
#     ready_dataset[f"{language}_support"] = ready_dataset["languages"].apply(lambda x: (language in x) if not pd.isna(x) else False)

In [34]:
# for genre in genres_to_include:
#     ready_dataset[f"is_{genre}"] = ready_dataset["genre"].apply(lambda x: (genre in x) if not pd.isna(x) else False)

In [ ]:
from sklearn import pipeline, impute, preprocessing

pipe = pipeline.Pipeline([
    ("games_achievements_imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
    ("refined_price_imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
    # languages
    ("languages_one_hot_imputer", preprocessing.MultiLabelBinarizer(classes=languages_to_include)),
    ("languages_one_hot", preprocessing.MultiLabelBinarizer(classes=languages_to_include)),
    # genres
    ("genres_one_hot", preprocessing.MultilabelBinarizer(classes=genres_to_include)),
    # developer
    ("developer_imputer", preprocessing.SimpleImputer(strategy="constant", fill_value="none")),
    ("developer_one_hot", preprocessing.OneHotEncoder()),
    # publisher
    ("publisher_imputer", preprocessing.SimpleImputer(strategy="constant", fill_value="none")),
    ("publisher_one_hot", preprocessing.OneHotEncoder()),
])

In [9]:
ready_dataset.isna().mean()

user_id                   0.000000
app_id                    0.000000
voted_up                  0.000000
votes_up                  0.000000
votes_funny               0.000000
weighted_vote_score       0.000000
playtime_forever          0.000000
playtime_at_review        0.000000
num_games_owned           0.000000
num_reviews               0.000000
review                    0.001539
unix_timestamp_created    0.000000
unix_timestamp_updated    0.000000
types                     0.027311
title_sgcd                0.027311
desc_snippet              0.027311
recent_reviews            0.108849
all_reviews               0.039099
release_date_sgcd         0.039181
developer                 0.027311
publisher                 0.027311
popular_tags              0.039099
game_details              0.029096
languages                 0.027311
n_game_achievements       0.239011
genre                     0.027388
game_description          0.039099
mature_content            0.790876
original_price_sgcd 